In [7]:
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm


## NOTE
The labels are `class, x, y, width, height`

We *REMOVE* `class` so that its just `x, y, width, height`

In [8]:
LABEL_DIR = '/Users/kuba/Documents/data/raw/face-detection-dataset/labels'
IMG_DIR = '/Users/kuba/Documents/data/raw/face-detection-dataset/images'

train_file_names = '/Users/kuba/projects/swvl/data-process/single-faces/single_face_train.txt'
test_file_names = '/Users/kuba/projects/swvl/data-process/single-faces/single_face_test.txt'


In [9]:
def make_paths(names_path, class_type):
    with open(names_path, 'r') as file_names:
        names = file_names.readlines()
        img_paths = [ IMG_DIR + "/" + class_type + "/" + name.strip() + ".jpg" for name in names]
        label_paths = [ LABEL_DIR + "/" + class_type + "/" + name.strip() + ".txt" for name in names]
    
    return img_paths, label_paths


In [10]:
train_img_paths, train_label_paths = make_paths(train_file_names, 'train')
test_img_paths, test_label_paths = make_paths(test_file_names, 'val')

In [11]:
def process_and_pad_data(img_paths, label_paths, save_location,  target_size=(224, 224)):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(target_size, antialias=True),  # Resize all images to same dimensions
    ])

    transform_normal =  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
    for f_img, f_label in tqdm(list(zip(img_paths, label_paths))):
        img = Image.open(f_img)
        img_tensor = transform(img)
        if img_tensor.size()[0] == 1:
            img_tensor = img_tensor.repeat(3, 1, 1)

        if img_tensor.size(0) == 4:
            #keep only the first 3 channels (RGB)
            img_tensor = img_tensor[:3, :, :]

        if img_tensor.size()[0] != 3:
            print(f_img)
            print(img_tensor.shape)

        img_tensor = transform_normal(img_tensor)

        with open(f_label, 'r') as f:
            label = f.readline().strip()
            bbox = torch.tensor([float(x) for x in label.split()][1:])

        name = f_img.split("/")[-1].split(".")[0]
        sav_name = save_location + "/" + name + ".pt"
        torch.save((img_tensor, bbox), sav_name)


In [12]:
process_and_pad_data(train_img_paths, train_label_paths, '/Users/kuba/Documents/data/raw/single-face-tensors/train')
process_and_pad_data(test_img_paths, test_label_paths, '/Users/kuba/Documents/data/raw/single-face-tensors/test')

100%|██████████| 1469/1469 [00:16<00:00, 88.53it/s]


___

Individal process

In [13]:
# f_img = f'/home/kuba/Documents/data/raw/me-img/raw/sun.jpeg'
# name = 'sun'


In [14]:
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize((224,224), antialias=True)  # Resize all images to same dimensions
# ])

# transform_normal =  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# img = Image.open(f_img)
# img_tensor = transform(img)
# img_tensor = transform_normal(img_tensor)

# torch.save(img_tensor, f'/home/kuba/Documents/data/raw/me-img/tensor/{name}.pt')